# Data Cleaning

This notebook executes the data cleaning part. 

In [2]:
# Required libraries
# Required libraries
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from pathlib import Path
import sys
import os 
import pandas as pd
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import datetime
import zipfile 
from timeit import default_timer as timer
import sqlalchemy as db
# Paths
sys.path.append(os.path.join(Path(os.getcwd()).parent))  
data_path = os.path.join(os.path.join(Path(os.getcwd()).parent), 'data')
data_per_day_path = os.path.join(os.path.join(Path(os.getcwd()).parent), 'data','data_per_day')


# create connection to sqlite database
db_path = os.path.join(data_path, 'database.db')
db_engine = db.create_engine('sqlite:///' + db_path)

In [2]:
# get the data folders file now
data_folders = [f for f in os.listdir(data_per_day_path) if not os.path.isfile(os.path.join(data_per_day_path, f))]
data_folders = [file for file in data_folders if '.' not in file]
data_folders = [os.path.join(data_per_day_path, x) for x in data_folders]
data_folders

['/data/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/SPY',
 '/data/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/EZU',
 '/data/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/VEA',
 '/data/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/EEM',
 '/data/Dropbox/Projects/financial_volatility/financial_volatility/data/data_per_day/IWM']

In [3]:
# get the csv file now
data_folder = data_folders[1]
table_name = data_folder[-3:]
csv_files = [f for f in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, f))]
csv_files = [file for file in csv_files if '.csv' in file]
csv_files = np.sort([os.path.join(data_folder, x) for x in csv_files])

In [4]:
data_folder[-3:]

'EZU'

In [5]:
def compute_returns(csv_file):
    data_df = pd.read_csv(csv_file)
    data_df.DT = pd.to_datetime(data_df.DT)
    data_df.sort_values(by=['DT'], inplace=True)
    data_df.index = data_df.DT
    data_df.drop(columns=['DT'],inplace=True)
    data_df = data_df.between_time('9:30', '16:00')
    data_df.reset_index(drop=False, inplace=True)

    # non zero quotes
    data_df = data_df.loc[(data_df.BID>0) & (data_df.BIDSIZ>0) & (data_df.ASK>0) & (data_df.ASKSIZ>0)]

    # autoselect exchange
    data_df['total_size'] = data_df.BIDSIZ + data_df.ASKSIZ
    #data_df = data_df.loc[data_df.EX == data_df.groupby(['EX']).sum().total_size.idxmax()]

    # delete negative spreads
    data_df = data_df.loc[data_df.ASK > data_df.BID]

    # mergeQuotesSameTimestamp
    ex = data_df.EX.values[0]
    sym_root = data_df.SYM_ROOT.values[0]
    data_df.drop(columns=['SYM_SUFFIX', 'total_size'], inplace=True)
    data_df = data_df.groupby(['DT']).median()
    data_df['EX'] = ex
    data_df['SYM_ROOT'] = sym_root
    data_df.reset_index(drop=False, inplace=True)

    # remove entries with spread > 50 * daily median spread
    data_df['SPREAD'] = data_df.ASK - data_df.BID
    data_df = data_df.loc[data_df['SPREAD'] < 50 * data_df['SPREAD'].median()]

    # remove outliers using the centered rolling window approach 
    def compute_diff(x):
        return x.values[window] - np.median(np.delete(x.values,window))

    window = 25
    data_df.sort_values(by=['DT'], inplace=True)
    data_df['SPREAD_DIFF'] = data_df.SPREAD.rolling(2*window+1, min_periods=2*window+1, center=True).apply(compute_diff)
    data_df = data_df.loc[(data_df['SPREAD_DIFF'] < 10 * data_df['SPREAD_DIFF'].mean()) | (data_df['SPREAD_DIFF'].isna())]
    data_df = data_df.reset_index(drop=True)
    
    # resample data to 15 minute level
    data_df.set_index(['DT'], inplace=True)
    data_df["MID"] = data_df.apply(lambda x: (x.ASK * x.ASKSIZ + x.BID * x.BIDSIZ) / (x.ASKSIZ + x.BIDSIZ), axis=1)
    data_df = data_df[['MID', 'SYM_ROOT']]
    df_resampled = data_df.resample('15min').ffill()
    df_resampled = df_resampled.append(pd.DataFrame(data_df[-1:].values, 
                                                    index=[df_resampled.index[-1] + datetime.timedelta(minutes=15)],columns=data_df.columns)) # get last observation that is not added by ffill

    # set new index and forward fill the price data
    df_resampled = df_resampled.iloc[1:,:] # observation at 9:30 is going to be NA
    new_index = pd.date_range(start=df_resampled.index[0].replace(hour=9, minute=45, second=0), periods=26, freq='15min') # index from 9:45 until 16:00
    df_resampled = df_resampled.reindex(new_index, method='ffill')
    df_resampled.reset_index(drop=False, inplace=True)
    
    # compute returns 
    df_resampled['RET'] = df_resampled.MID.pct_change().apply(np.vectorize(lambda x: np.log(1+x)))
    df_resampled = df_resampled.iloc[1:,:] # first return is NA
    df_resampled.rename(columns={'index': 'DT'}, inplace = True)
    return df_resampled[['DT', 'RET']]

In [6]:
%%time
from joblib import Parallel, delayed
    
df_data_all_days = Parallel(n_jobs=14)(delayed(compute_returns)(i) for i in csv_files)

CPU times: user 433 ms, sys: 55.1 ms, total: 488 ms
Wall time: 7min 51s


In [8]:
for df in df_data_all_days:
    df.to_sql(data_folder[-3:], db_engine, index=False, if_exists='append')

In [12]:
table_names = db.inspect(db_engine).get_table_names()

In [13]:
for table_name in table_names:
    data = pd.read_sql(f"select * from {table_name}", db_engine)
    if 'db_returns' in locals():
        db_returns[table_name] = data.RET.values
    else:
        db_returns = pd.DataFrame(data.RET.values, index=data.DT, columns=[table_name])
db_returns.reset_index(drop=False, inplace=True)
db_returns.to_sql('returns',db_engine, index=False, if_exists='replace')

In [ ]:
plt.figure(figsize=(20, 10))
    plt.plot(resample, RV_av,color='#9932CC',linewidth=2)
    plt.savefig(os.path.join(results_path, 'RV_resampling_decay'), dpi=400, facecolor='aliceblue', edgecolor='k',bbox_inches='tight')
    plt.show()